#Reading CSV Files

In [ ]:
print("hello Jiayi and Amanda")

In [ ]:
# import the libraries we will need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
test_df = pd.read_csv('./test.csv')
train_df = pd.read_csv('./train.csv')

In [ ]:
test_df.columns

In [ ]:
train_df.head()

In [ ]:
train_df.columns

In [ ]:
train_df['town'].unique()

In [ ]:
train_df['flat_type'].unique()

#DATA CLEANING

## Looking for columns with null values

In [ ]:
train_df.isnull().sum()[train_df.isnull().sum() > 0]

In [ ]:
# # Replacing null values with '0' in the columns 'Mall_Within_500m', 'Mall_Within_1km', 'Mall_Within_2km'.
train_df.fillna(value = {'Mall_Within_500m': 0, 'Mall_Within_1km': 0, 'Mall_Within_2km': 0}, inplace = True)

In [ ]:
train_df[['Mall_Within_500m', 'Mall_Within_1km', 'Mall_Within_2km']].head(10)

In [ ]:
train_df.isnull().sum()[train_df.isnull().sum() > 0]

In [ ]:
train_df.dropna(subset = ['Mall_Nearest_Distance'], inplace = True)

In [ ]:
train_df.fillna(value = {'Hawker_Within_500m': 0, 'Hawker_Within_1km': 0, 'Hawker_Within_2km': 0}, inplace = True)

In [ ]:
train_df.isnull().sum()[train_df.isnull().sum() > 0]

In [ ]:
train_df

#DATA TYPES

## Convert data to the correct Dtype

In [ ]:
train_df.info()

In [ ]:
train_df['Tranc_YearMonth'] = pd.to_datetime(train_df['Tranc_YearMonth'])

In [ ]:
train_df.info()

#GET_DUMMIES

## Convert to dummies for categorical columns: Flat, storey range, town

In [ ]:
flat_dummies = pd.get_dummies(train_df.flat_type, prefix='flat_type',drop_first=True)

In [ ]:
flat_dummies.sample(n=5,random_state=1)

In [ ]:
town_dummies = pd.get_dummies(train_df.town, prefix='town',drop_first=True)

In [ ]:
town_dummies.sample(n=5, random_state=1)

In [ ]:
storey_dummies = pd.get_dummies(train_df.storey_range, prefix='storey_range',drop_first=True)

In [ ]:
storey_dummies.sample(n=5, random_state=1)

In [ ]:
storey_dummies.columns

In [ ]:
town_dummies.columns

In [ ]:
train_dummies = pd.concat([train_df, town_dummies, storey_dummies], axis=1)
train_dummies.sample(n=5, random_state=1)

#Correlation 

## Check if there are any correlations between independent variables

In [ ]:
train_dummies_corr = train_dummies[[
      'town_BEDOK','town_BISHAN','town_BUKIT BATOK','town_BUKIT PANJANG', 'town_BUKIT TIMAH', 'town_CENTRAL AREA',
       'town_CHOA CHU KANG', 'town_CLEMENTI', 'town_GEYLANG', 'town_HOUGANG',
       'town_JURONG EAST', 'town_JURONG WEST', 'town_KALLANG/WHAMPOA',
       'town_MARINE PARADE', 'town_PASIR RIS', 'town_PUNGGOL',
       'town_QUEENSTOWN', 'town_SEMBAWANG', 'town_SENGKANG', 'town_SERANGOON',
       'town_TAMPINES', 'town_TOA PAYOH', 'town_WOODLANDS', 'town_YISHUN',
       'floor_area_sqm', 'lease_commence_date', 'resale_price', 'Tranc_Year', 'Tranc_Month',
       'Mall_Nearest_Distance', 'Hawker_Nearest_Distance', 'mrt_nearest_distance',
       'storey_range_01 TO 05', 'storey_range_04 TO 06',
       'storey_range_06 TO 10', 'storey_range_07 TO 09',
       'storey_range_10 TO 12', 'storey_range_11 TO 15',
       'storey_range_13 TO 15', 'storey_range_16 TO 18',
       'storey_range_16 TO 20', 'storey_range_19 TO 21',
       'storey_range_21 TO 25', 'storey_range_22 TO 24',
       'storey_range_25 TO 27', 'storey_range_26 TO 30',
       'storey_range_28 TO 30', 'storey_range_31 TO 33',
       'storey_range_31 TO 35', 'storey_range_34 TO 36',
       'storey_range_36 TO 40', 'storey_range_37 TO 39',
       'storey_range_40 TO 42', 'storey_range_43 TO 45',
       'storey_range_46 TO 48', 'storey_range_49 TO 51'
       ]].corr()

In [ ]:
# Create positive correlation matrix and also filtering out strong negative correlations
corr_df = train_dummies_corr.abs()

In [ ]:
# Create and apply mask
mask = np.triu(np.ones_like(corr_df, dtype=bool))

In [ ]:
# Replacing the DataFrame with the mask where the mask has a True value in the upper triangle
corr_df = corr_df.mask(mask)
corr_df

In [ ]:
# Finding columns that have correlation greater than 0.7 (which is our threshold)
to_drop = [column for column in corr_df.columns if any(corr_df[column] > 0.65)]
print(to_drop)

#EDA

In [ ]:
# Histogram for Resale price
import plotly.express as px
fig = px.histogram(train_dummies, x="resale_price",
                   title='Distribution of Resale Price',
                   opacity=0.8,
                   color_discrete_sequence=['green']
                   )

fig.update_layout(
    xaxis_title='Resale Price ($)',  # x-axis label
    yaxis_title='Number of Transactions'      # y-axis label
)


fig.show()

In [ ]:
# Floor area vs Resale price
fig = px.scatter(train_dummies, x="floor_area_sqm", y="resale_price", color="floor_area_sqm")

fig.update_layout(
    title='Resale Prices Across Different Floor Area (sqm)',
    xaxis_title='Floor Area (sqm)',  # x-axis label
    yaxis_title='Resale Price'      # y-axis label
)


fig.show()

In [ ]:
# Resale prices across different flat types

mean_resale_price = train_dummies.groupby('flat_type', as_index=False)['resale_price'].mean()
fig = px.histogram(mean_resale_price, x="flat_type", y="resale_price", 
                   title='Resale Prices Across Different Flat Types',
                   opacity=0.8,
                   color_discrete_sequence=['blue']
                   )
fig.update_layout(
    xaxis_title='Flat Type',  # x-axis label
    yaxis_title='Average Resale Price ($)'      # y-axis label
)


fig.show()

In [ ]:
import matplotlib.pyplot as plt

# Group by 'Tranc_YearMonth' and calculate the mean of 'resale_price'
mean_resale_price = train_dummies.groupby('Tranc_YearMonth')['resale_price'].mean()

# Plot the results using the correct x and y values from the computed series
plt.plot(mean_resale_price.index, mean_resale_price.values)
plt.xticks(rotation=45)
plt.xlabel('Month')  # Label for the x-axis
plt.ylabel('Average Resale Price ($)')  # Label for the y-axis
plt.title('Average Resale Price by Month')  # Title for the plot
plt.tight_layout()
plt.show()

In [ ]:
# Resale prices across different storey range
mean_resale_price = train_dummies.groupby('storey_range', as_index=False)['resale_price'].mean()
fig = px.histogram(mean_resale_price, x="storey_range", y="resale_price", 
                   title='Resale prices across different storey range',
                   opacity=0.8,
                   color_discrete_sequence=['green']
                   )
fig.update_layout(
    xaxis_title='Flat Type',  # x-axis label
    yaxis_title='HDB Storey Range'      # y-axis label
)


fig.show()

#EDA (Based on Region)

In [ ]:
train_df['town'].unique()

In [ ]:
# Fixing the KeyError by correctly filtering the dataframe by logical conditions instead of indexing
# Assigning regions based on town names
central_region_towns = ['KALLANG/WHAMPOA', 'BUKIT MERAH', 'TOA PAYOH', 'ANG MO KIO', 'BISHAN', 'QUEENSTOWN', 'BUKIT TIMAH', 'CENTRAL AREA', 'MARINE PARADE']
north_region_towns = ['YISHUN', 'HOUGANG', 'SENGKANG', 'SERANGOON', 'WOODLANDS', 'SEMBAWANG']
west_region_towns = ['BUKIT PANJANG', 'JURONG WEST', 'CHOA CHU KANG', 'CLEMENTI', 'JURONG EAST','BUKIT BATOK']
east_region_towns = ['GEYLANG', 'TAMPINES', 'PASIR RIS', 'BEDOK', 'PUNGGOL']

# Filtering data (using DataFrame.isin method to avoid KeyErrors)
town_central_region = train_df[train_df['town'].isin(central_region_towns)]
town_north_region = train_df[train_df['town'].isin(north_region_towns)]
town_west_region = train_df[train_df['town'].isin(west_region_towns)]
town_east_region = train_df[train_df['town'].isin(east_region_towns)]

In [ ]:
def assign_region(row):
    if row['town'] in central_region_towns:
        return 'Central'
    elif row['town'] in north_region_towns:
        return 'North'
    elif row['town'] in west_region_towns:
        return 'West'
    elif row['town'] in east_region_towns:
        return 'East'
    else:
        return 'Unknown'

# Applying the function to create a 'region' column
train_dummies['region'] = train_dummies.apply(assign_region, axis=1)

In [ ]:
# Resale prices across different regions in Singapore

mean_resale_price = train_dummies.groupby('region', as_index=False)['resale_price'].mean()
fig = px.histogram(mean_resale_price, x="region", y="resale_price", 
                   title='Resale prices across different regions in Singapore',
                   opacity=0.8,
                   color_discrete_sequence=['blue']
                   )
fig.update_layout(
    xaxis_title='Region',  # x-axis label
    yaxis_title='Resale Price ($)'      # y-axis label
)

fig.show()

In [ ]:
central = train_dummies[train_dummies['region'] == 'Central']
central

In [ ]:
# Plotting by Central region in Singapore
mean_resale_price_central = central.groupby('Tranc_YearMonth', as_index=False)['resale_price'].mean()

# Plot the results using the correct x and y values from the computed series
plt.plot(mean_resale_price_central.Tranc_YearMonth, mean_resale_price_central.resale_price)
plt.xticks(rotation=45)
plt.xlabel('Tranc_YearMonth')  # Label for the x-axis
plt.ylabel('Mean_Resale_Price')  # Label for the y-axis
plt.title('Tranc_Year vs Mean_Resale_Price (Central Region)')  # Title for the plot
plt.tight_layout()
plt.show()

In [ ]:
west = train_dummies[train_dummies['region'] == 'West']
west

In [ ]:
# Plotting by West region in Singapore
mean_resale_price_west = west.groupby('Tranc_YearMonth', as_index=False)['resale_price'].mean()

# Plot the results using the correct x and y values from the computed series
plt.plot(mean_resale_price_west.Tranc_YearMonth, mean_resale_price_west.resale_price)
plt.xticks(rotation=45)
plt.xlabel('Tranc_YearMonth')  # Label for the x-axis
plt.ylabel('Mean_Resale_Price')  # Label for the y-axis
plt.title('Tranc_Year vs Mean_Resale_Price (West Region)')  # Title for the plot
plt.tight_layout()
plt.show()

In [ ]:
north = train_dummies[train_dummies['region'] == 'North']
north

In [ ]:
# Plotting by North region in Singapore
mean_resale_price_north = north.groupby('Tranc_YearMonth', as_index=False)['resale_price'].mean()

# Plot the results using the correct x and y values from the computed series
plt.plot(mean_resale_price_north.Tranc_YearMonth, mean_resale_price_north.resale_price)
plt.xticks(rotation=45)
plt.xlabel('Tranc_YearMonth')  # Label for the x-axis
plt.ylabel('Mean_Resale_Price')  # Label for the y-axis
plt.title('Tranc_Year vs Mean_Resale_Price (North Region)')  # Title for the plot
plt.tight_layout()
plt.show()

In [ ]:
east = train_dummies[train_dummies['region'] == 'East']
east

In [ ]:
# Plotting by East region in Singapore
mean_resale_price_east = east.groupby('Tranc_YearMonth', as_index=False)['resale_price'].mean()

# Plot the results using the correct x and y values from the computed series
plt.plot(mean_resale_price_east.Tranc_YearMonth, mean_resale_price_east.resale_price)
plt.xticks(rotation=45)
plt.xlabel('Tranc_YearMonth')  # Label for the x-axis
plt.ylabel('Mean_Resale_Price')  # Label for the y-axis
plt.title('Tranc_Year vs Mean_Resale_Price (East Region)')  # Title for the plot
plt.tight_layout()
plt.show()

#EDA (Based on SuperMalls)

In [ ]:
# <THIS IS JUST A NOTE>
# List of Supermalls near residential area in Singapore:

# Central Region: ION Orchard, Marina Bay Sands Shoppes, Suntec City, Plaza Singapura, Bugis Junction, BV Mall
# East Region: Tampines 1, Bedok Mall, Eastpoint Mall, Parkway Parade, Siglap Centre
# North Region: Causeway Point, Nex, Northpoint City, Sembawang Shopping Centre, Vista Point
# West Region: Vivocity, Westgate, JEM, The Clementi Mall, Jurong Point 

In [ ]:
# Coordinates (Latitude and Longtitude)

coordinates_central = {'Supermall': ['ION Orchard','Marina Bay Sands Shoppes','Suntec City', 'Plaza Singapura', 'Bugis Junction', 'BV Mall'],
                      'SLatitude': [1.30411, 1.28397, 1.29552, 1.30029, 1.29960, 1.2833],
                      'SLongitude': [103.83197, 103.85914, 103.85846, 103.84482, 103.85558, 103.8500]}

coordinates_north = {'Supermall': ['Causeway Point', 'Nex', 'Northpoint City', 'Sembawang Shopping Centre', 'Vista Point'],
                     'SLatitude': [1.43634, 1.35076, 1.42958, 1.44176, 1.43156],
                     'SLongitude': [103.78603, 103.87202, 103.83580, 103.82455, 103.79390]}

coordinates_west = {'Supermall': ['Vivocity', 'Westgate', 'JEM', 'The Clementi Mall', 'Jurong Point'],
                    'SLatitude': [1.26499, 1.33460, 1.33278, 1.31508, 1.34000],
                    'SLongitude': [103.82324, 103.74278, 103.74324, 103.76437, 103.70678]}

coordinates_east = {'Supermall': ['Tampines 1', 'Bedok Mall', 'Eastpoint Mall', 'Parkway Parade', 'Siglap Centre'],
                    'SLatitude': [1.35431, 1.32492, 1.34295, 1.30164, 1.31274],
                    'SLongitude': [103.94544, 103.92931, 103.95311, 103.90529, 103.92332]}                 

In [ ]:
import pandas as pd

# Creating DataFrames from the dictionaries above
coordinates_central = pd.DataFrame(coordinates_central)
coordinates_north = pd.DataFrame(coordinates_north)
coordinates_west = pd.DataFrame(coordinates_west)
coordinates_east = pd.DataFrame(coordinates_east)

In [ ]:
# Concatenating the DataFrames
coordinates_of_mall = pd.concat([coordinates_central, coordinates_north, coordinates_west, coordinates_east])
coordinates_of_mall

In [ ]:
# Drawing out the relevant Data:
coordinates_of_resi = train_dummies[['id','Latitude', 'Longitude']]
coordinates_of_resi

In [ ]:
# Function to calculate distance between two points on Earth

def haversine_distance(lat1, lon1, lat2, lon2):
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6371 * c  # Multiply by Earth's radius in kilometers
    return km

 
 # Calculate all distances at once using broadcasting

lat1 = coordinates_of_resi['Latitude'].values[:, np.newaxis]
lon1 = coordinates_of_resi['Longitude'].values[:, np.newaxis]
lat2 = coordinates_of_mall['SLatitude'].values
lon2 = coordinates_of_mall['SLongitude'].values

distances = haversine_distance(lat1, lon1, lat2, lon2)

# Find the nearest mall for each residential area
tmin_distance_indices = np.argmin(distances, axis=1)

# Prepare result DataFrame
min_distance_df = pd.DataFrame({
    #"Residential ID": coordinates_of_resi['id'],
    "Nearest Supermall": coordinates_of_mall['Supermall'].values[tmin_distance_indices],
    "Distance (km)": distances[np.arange(distances.shape[0]), tmin_distance_indices]
})

# Print results
print(min_distance_df)

In [ ]:
min_distance_df

In [ ]:
# Concatenate Dataframes 
train_dummies_1 = pd.concat([train_dummies, min_distance_df], axis=1)
train_dummies_1

In [ ]:
# Floor area vs Resale price
fig = px.scatter(train_dummies_1, x="Distance (km)", y="resale_price", color="Distance (km)")

fig.update_layout(
    title='Resale Prices VS. Distance from Supermall',
    xaxis_title='Distance from Supermall',  # x-axis label
    yaxis_title='Resale Price'      # y-axis label
)


fig.show()

In [ ]:
def categorize_distance(distance):
    if pd.isna(distance):
        return "Unknown"
    if distance < 1 or distance > 8:
        return "Out of range"
    return f"{int(distance)} km"

# Apply the updated categorize_distance function to the 'Distance (km)' column
train_dummies_1['dist_category'] = train_dummies_1['Distance (km)'].apply(categorize_distance)

# Display the updated DataFrame
train_dummies_1

In [ ]:
# Resale prices based on location away from Supermall
mean_resale_price_dist = train_dummies_1.groupby('Distance (km)', as_index=False)['resale_price'].mean()
fig = px.histogram(mean_resale_price_dist, x="Distance (km)", y="resale_price", 
                   title='Resale prices based on location away from Supermall',
                   opacity=0.8,
                   color_discrete_sequence=['navy']
                   )
fig.update_layout(
    xaxis_title='Distance (km)',  # x-axis label
    yaxis_title='HDB Resale Price'      # y-axis label
)


fig.show()

In [ ]:
# Resale prices based on location away from Supermall
mean_resale_price_dist = train_dummies_1.groupby('dist_category', as_index=False)['resale_price'].mean()
fig = px.histogram(mean_resale_price_dist, x="dist_category", y="resale_price", 
                   title='Resale prices based on location away from Supermall',
                   opacity=0.8,
                   color_discrete_sequence=['cyan']
                   )
fig.update_layout(
    xaxis_title='Distance (km)',  # x-axis label
    yaxis_title='HDB Resale Price'      # y-axis label
)


fig.show()

#Modeling - Linear Regression

In [ ]:
# Include variables: town, floor_area_sqm, lease_commence_date, resale_price, Tranc_Year, Tranc_Month, Mall_Nearest_Distance, Hawker_Nearest_Distance, mrt_nearest_distance

feature_cols = [
      'town_BEDOK','town_BISHAN','town_BUKIT BATOK','town_BUKIT PANJANG', 'town_BUKIT TIMAH', 'town_CENTRAL AREA',
       'town_CHOA CHU KANG', 'town_CLEMENTI', 'town_GEYLANG', 'town_HOUGANG',
       'town_JURONG EAST', 'town_JURONG WEST', 'town_KALLANG/WHAMPOA',
       'town_MARINE PARADE', 'town_PASIR RIS', 'town_PUNGGOL',
       'town_QUEENSTOWN', 'town_SEMBAWANG', 'town_SENGKANG', 'town_SERANGOON',
       'town_TAMPINES', 'town_TOA PAYOH', 'town_WOODLANDS', 'town_YISHUN',
       'floor_area_sqm', 'lease_commence_date', 'Tranc_Year', 'Tranc_Month',
       'Mall_Nearest_Distance', 'Hawker_Nearest_Distance', 'mrt_nearest_distance'
       ]

In [ ]:
# Include variables: town, floor_area_sqm, lease_commence_date, resale_price, Tranc_Year, Tranc_Month, Mall_Nearest_Distance, Hawker_Nearest_Distance, mrt_nearest_distance
# Include more variables: storey_range

feature_cols_1 = [
      'town_BEDOK','town_BISHAN','town_BUKIT BATOK','town_BUKIT PANJANG', 'town_BUKIT TIMAH', 'town_CENTRAL AREA',
       'town_CHOA CHU KANG', 'town_CLEMENTI', 'town_GEYLANG', 'town_HOUGANG',
       'town_JURONG EAST', 'town_JURONG WEST', 'town_KALLANG/WHAMPOA',
       'town_MARINE PARADE', 'town_PASIR RIS', 'town_PUNGGOL',
       'town_QUEENSTOWN', 'town_SEMBAWANG', 'town_SENGKANG', 'town_SERANGOON',
       'town_TAMPINES', 'town_TOA PAYOH', 'town_WOODLANDS', 'town_YISHUN',
       'floor_area_sqm', 'lease_commence_date', 
       'Tranc_Year', 'Tranc_Month',
       'Mall_Nearest_Distance', 'Hawker_Nearest_Distance', 'mrt_nearest_distance',
       'storey_range_01 TO 05', 'storey_range_04 TO 06',
       'storey_range_06 TO 10', 'storey_range_07 TO 09',
       'storey_range_10 TO 12', 'storey_range_11 TO 15',
       'storey_range_13 TO 15', 'storey_range_16 TO 18',
       'storey_range_16 TO 20', 'storey_range_19 TO 21',
       'storey_range_21 TO 25', 'storey_range_22 TO 24',
       'storey_range_25 TO 27', 'storey_range_26 TO 30',
       'storey_range_28 TO 30', 'storey_range_31 TO 33',
       'storey_range_31 TO 35', 'storey_range_34 TO 36',
       'storey_range_36 TO 40', 'storey_range_37 TO 39',
       'storey_range_40 TO 42', 'storey_range_43 TO 45',
       'storey_range_46 TO 48', 'storey_range_49 TO 51'
       ]

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:

# Define a function that accepts a list of features in a DataFrame and returns the RMSE for a test dataset.

def train_test_rmse(df, features):
    X = df[features]
    y = df['resale_price']

    X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=123)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    linreg = LinearRegression()
    linreg.fit(X_train, y_train)

    y_pred = linreg.predict(X_test)
    
    # Create your scatter plot
    plt.plot(y_test, y_pred, 'o', markersize=6, label='Data Points', color='blue')

    # Fit a line to the data
    m, b = np.polyfit(y_test, y_pred, 1)
    plt.plot(y_test, m*y_test + b, color='red', label=f'Fit: y = {m:.2f}x + {b:.2f}')

    # Add grid
    plt.grid(True, linestyle='--', alpha=0.2)

    # Add labels and title
    plt.xlabel('Actual Resale Price', fontsize=12)
    plt.ylabel('Predicted Resale Price', fontsize=12)
    plt.title('Actual vs Predicted Resale Price', fontsize=14)

    # Show the plot
    plt.tight_layout()
    plt.show()

    print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print('R2 Score:', metrics.r2_score(y_test, y_pred))
    print(linreg.coef_)
    print(linreg.intercept_)
    print(list(zip(features, linreg.coef_)))

## Use 'feature_cols' for Model 1  as trial

In [ ]:

train_test_rmse(train_dummies, feature_cols)

In [ ]:
train_dummies

In [ ]:
print(train_dummies[feature_cols].corrwith(train_dummies['resale_price']))

## Use 'feature_cols_1' as improvement for Model 2  (final)

In [ ]:
# Use 'feature_cols_1' for Model 2
train_test_rmse(train_dummies, feature_cols_1)

#Fit using Test Data

In [ ]:
test_town_dummies = pd.get_dummies(test_df.town, prefix='town',drop_first=True)
test_storey_dummies = pd.get_dummies(test_df.storey_range, prefix='storey_range',drop_first=True)


In [ ]:
test_dummies = pd.concat([test_df, test_town_dummies, test_storey_dummies], axis=1)

In [ ]:
test_dummies.sample(n=5, random_state=1)

In [ ]:
test_df.isnull().sum()[test_df.isnull().sum() > 0]

In [ ]:
test_df.fillna(value = {'Mall_Within_500m': 0, 'Mall_Within_1km': 0, 'Mall_Within_2km': 0}, inplace = True)
test_df.dropna(subset = ['Mall_Nearest_Distance'], inplace = True)
test_df.fillna(value = {'Hawker_Within_500m': 0, 'Hawker_Within_1km': 0, 'Hawker_Within_2km': 0}, inplace = True)

In [ ]:
test_df.isnull().sum()[test_df.isnull().sum() > 0]

In [ ]:
expected_storey_range_cols = ['storey_range_31 TO 35', 'storey_range_36 TO 40']

# Add missing storey range columns if they are not present in test_dummies
for col in expected_storey_range_cols:
    if col not in test_dummies.columns:
        test_dummies[col] = False  # Assuming default value is False

In [ ]:
test_dummies

In [ ]:

X = train_dummies[feature_cols_1]
y = train_dummies['resale_price']
X_kaggle = test_dummies[feature_cols_1]

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=123)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

linreg = LinearRegression()
linreg.fit(X_train, y_train)

y_pred = linreg.predict(X_test)



In [ ]:
y_pred

In [ ]:
X_kaggle = scaler.transform(X_kaggle)

y_kaggle = linreg.predict(X_kaggle)

In [ ]:
y_kaggle

In [ ]:
# Slicing ID column first
test_id = test_dummies['id']

In [ ]:
# Creating a DataFrame from the test id that was sliced earlier
result = pd.DataFrame(test_id)
result

In [ ]:
# Adding the prediction from test data alongside the id
result['Predicted'] = y_kaggle
result

In [ ]:
# Save the csv file to an existing directory in the current environment
result.to_csv('result_kaggle.csv', index = False)

#Forecasting

In [ ]:
train_dummies['Tranc_YearMonth'].sort_values()

In [ ]:
train_dummies.head()

In [ ]:
train_forecast = train_dummies.set_index("Tranc_YearMonth")

In [ ]:
train_forecast.head()

In [ ]:
monthly_sales = train_forecast.resample("M")["resale_price"].mean()
monthly_sales.head()

In [ ]:
train_forecast["resale_price"].plot();

In [ ]:
train = monthly_sales["2012":"2019"].astype(float) # 2 full years
test = monthly_sales["2020":"2021"].astype(float) # most of 2015

train.head()

In [ ]:
train.plot(label="train")
test.plot(label="test")
plt.legend()
plt.show()

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

#order=(1, 0, 1) = p,d,q

model = ARIMA(train, order=(10, 0, 10)).fit()

y_pred = model.predict(start="2020-01", end="2021-04")

In [ ]:
# Define the 'ax' variable when plotting
fig, ax = plt.subplots()

train.plot(label="train", ax=ax)
test.plot(label="test", ax=ax)
y_pred.plot(label="ARIMA forecast", ax=ax)
ax.legend()

plt.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8d8e525f-edc3-44db-8702-1e50c5f3157c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>